In [2]:
import pandas as pd
import json
from sqlalchemy import create_engine
import zipfile
import tarfile
import bz2
import sys
import os

### As test data:
#### load only two files from every hour folder from the first day of the month

In [3]:
tweet_data = []

for folder in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/")):
    if not folder.startswith('.'):
        print('load folder', folder)
        
        t = 0
        for file in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder)):
            
            t += 1
            if file.endswith('.bz2'):
                
                print(t)
                print('load file', file)

                with bz2.open("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder + "/" + file, "rt") as bzinput:


                    for i, line in enumerate(bzinput):
                        tweet = json.loads(line)
                        tweet_data.append(tweet)

            #include break statement to test
            # load only 2 files from each hour folder 
                if t == 2: break


load folder 00
1
load file 29.json.bz2
2
load file 30.json.bz2
load folder 01
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 02
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 03
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 04
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 05
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 06
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 07
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 08
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 09
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 10
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 11
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 12
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 13
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 14
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 15
1
load file 00.json.bz2
2
load file 01.j

In [3]:
len(tweet_data)

250819

### Lookup tables

In [4]:
# lookup table country names in different languages - country codes
df_c_multilingu = pd.read_csv('./data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV')
df_c_multilingu.head()

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
0,AF,AFRIKAANS,AD,AND,20,Andorra
1,AF,AFRIKAANS,AE,ARE,784,Verenigde Arabiese Emirate
2,AF,AFRIKAANS,AF,AFG,4,Afganistan
3,AF,AFRIKAANS,AG,ATG,28,Antigua en Barbuda
4,AF,AFRIKAANS,AI,AIA,660,Anguilla


In [15]:
# lookup table country codes - coordinates
# file is from googlemaps

df_co_from_cc = pd.read_csv('./data/location_ref/countrycodes_coord.csv')
# remove the second decimal places after the dot for valid latitude and logitude notation
df_co_from_cc['latitude'] = df_co_from_cc['latitude'].str.split('.').str[:-1].str.join('.')
df_co_from_cc['longitude'] = df_co_from_cc['longitude'].str.split('.').str[:-1].str.join('.')

df_co_from_cc.head(20)

,country,latitude,longitude
0,AD,42.546,1.601
1,AE,23.424,53.847
2,AF,3.393,67.709
3,AG,17.060,-61.796
4,AI,18.220,-63.068
5,AL,41.153,20.168
6,AM,40.069,45.038
7,AN,12.226,-69.060
8,AO,-11.202,17.873
9,AQ,-75.250,-71


#### Google Maps uses a Mercator projection based on the World Geodetic System (WGS) 1984 geographic coordinate system (datum) == 3857

WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS  == EPSG 4326m

### Get list of country names multilingu:

In [6]:
# filter for languages 
#https://www.loc.gov/standards/iso639-2/php/code_list.php
lang_4 = ['DE', 'FR', 'ES', 'SV']   

# EU + norway 
lang_european_un = ['DE', 'FR', 'HR', 'CS', 'DA', 'ET', 'FI', 
                    'EL',  'HU', 'GA', 'GL', 'IT', 'LT', 'LV', 'LB',
                    'SK', 'SL', 'NL', 'PL', 'PT', 'RO', 'SV', 'NN','ES', 'SV']

# assign current used languages list to "lang"
lang = lang_european_un

sel = df_c_multilingu[df_c_multilingu['LANG'].isin(lang)]
# # test
sel[sel['COUNTRY_NAME']=='Man']

c_nam_list = sel['COUNTRY_NAME'].to_list()

# remove problem causing country names 
c_nam_list.remove('Man')

# get unique country names
c_nam_list_unique = list(set(c_nam_list))
len(c_nam_list), len(c_nam_list_unique)

(5726, 3364)

### Functions

In [9]:
def get_from_country_coords(country_code) -> str:
    
    """Function to retrieve country coordinates from ISO country codes"""
    
    country = df_co_from_cc[df_co_from_cc['country'] == country_code]
    (lat, long) = (country['latitude'].to_list()[0], country['longitude'].to_list()[0])
    
    # Get country coords
    return f'{lat}, {long}'

In [10]:
# test
get_from_country_coords('IN')

'20.593, 7.896'

In [11]:
def get_to_country_coords(c_name_multilingu) -> str:
    
    """Function to retrieve to-coordinates from country names that appear in the tweet text"""
    # assign detected country name to countrycode:
    
    code = (df_c_multilingu.loc[df_c_multilingu['COUNTRY_NAME'] == c_name_multilingu]['COUNTRY_ALPHA2_CODE'].unique())[0]
    # get coordinates for countrycode from lookup table via function:
    coords = get_from_country_coords(code)
    
    return coords # both coordinates as string

In [12]:
# test
#get_to_country_coords('Chine')

In [13]:


def get_coordinates(bounding_coords, city_name, user_location) -> str:
    if bounding_coords:
        return bounding_coords
    elif city_name:
        # Lookup city name coordinates
        return '79.125472, 28.982011'
    elif user_location:
        # Lookup user location
        return '79.125472, 28.982011'
    elif country_code:
        # Lookup country code
        return get_from_country_coords(country_code)



### Write to postgres

In [12]:
# Declare postgres config
HOST = 'localhost'
USERNAME = 'postgres'
PORT = '5432'
DB = 'tweets_multilingu'
PASSWORD = 'postgres'

# Create a postgres connection and assign it to 'engine' variable
engine = create_engine(f'postgres://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [14]:
def extract_url(urls) -> str:
    
    """ Function to extract the url from the entities dict of the tweet json"""
    
    found_url = ''
    url_given = urls
    
    if url_given != None:
        for url in url_given:
            found_url = url.get('url')
            
    return found_url

In [14]:
def load(data):
    """ Loads transformed tweeets into the Postgres database
    Parameters:
    ------------
    tweets: List of tweet dictionaries that were retrieved from the Twitter API """

    insert_query = """INSERT INTO tweets_test (
        tweet_id, 
        timestamp_ms,
        tweet, 
        url_in_tweet, 
        user_url, 
        user_location, 
        user_followers_count, 
        user_verified, 
        user_statuses_count,
        user_id,
        user_created_at,

        lang,  
        from_coords,
        from_country_coords,
        to_country_coords,
        tweet_country_code)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    
    counter = 0 
    counter_loc = 0 
    counter_country_ment = 0 
    # loop through list of jsons 
    for j in data:
        # check for valid entries and if tweet json contains any location information 
        valid_entry = j.get('created_at')
        user_loc = j.get('user', {}).get('location')
        place_attr = j.get('place')
        user_url = j.get('user', {}).get('url')
        urls = j.get('entities', {}).get('urls')
        
        # conditions to still be included ! 
        follower_c = j.get('user', {}).get('followers_count')
        ver = j.get('user', {}).get('verified')
        
        counter += 1
        
        if (valid_entry and len(urls) != 0 and user_url) and (user_loc or place_attr) and (ver or follower_c > 10000):
            
            #print(ver, follower_c)

            counter_loc += 1 
            
            # loop through multilingual country list 
            for c_name_multi in c_nam_list_unique:
                
                
                # check if one of the country names exist in tweet text
                if c_name_multi in j['text']:
                    counter_country_ment += 1
                    print(c_name_multi)

                    # get user dict
                    user = j.get('user')
                    # ger user_location if exits
                    user_location = user.get('location')
                            
                    # get place dict if exists and extract first coordinate of the bounding box
                    place = j.get('place')
                    country_code = ''
                    tweet_country_code = ''
                    bounding_coords = ''
                    city_name = ''
                    from_coords = ''
                    from_country_coords = ''

                    if place != None:
                        country_code = place.get('country_code')
                        tweet_country_code = place.get('country_code')
                        
                        bounding_coords = ','.join(list(map(lambda n: str(n), place.get('bounding_box').get('coordinates')[0][0])))
                        
                        city_name = place.get('full_name')
                        from_coords = get_coordinates(bounding_coords, city_name, user_location)
                        from_country_coords = get_from_country_coords(country_code)
                        
                    to_country_coords = get_to_country_coords(c_name_multi)
                    print(to_country_coords)
                    
                    # get url mentionend in tweet from entities dict
                    found_url = extract_url(j.get('entities').get('urls'))
                    

                    # insert values into postgres DB                             
                    engine.execute(insert_query, (
                        j['id'], #id
                        j['timestamp_ms'], #timestamp_ms
                        j['text'], #tweet
                        found_url, #url_in_tweet
                        user['url'], #user_url
                        user_location, #user_location
                        user['followers_count'], #user_followers_count
                        user['verified'], #user_verified
                        user['statuses_count'], #user_statuses_count
                        user['id'], #user_id
                        user['created_at'], #user_created_at
                        
                        j['lang'], #lang
                        from_coords, #from_coords
                        from_country_coords, #from_country_coords
                        to_country_coords, #to_country_coords
                        tweet_country_code #tweet_country_code
                        )
         
                    )
    
    
    print('scanned jsons :', counter, 
          '\n with some location information and user url given :', counter_loc,
          '\n with some location information, user url and country name mentioned :',counter_country_ment)
    
     

In [18]:
load(tweet_data)

Granada
12.262.776, -61.604.171
Hong Kong
22.396.428, 114.109.497
Japan
36.204.824, 138.252.924
China
3.586.166, 104.195.397
Australia
-25.274.398, 133.775.136
Japan
36.204.824, 138.252.924
Peru
-9.189.967, -75.015.152
França
46.227.638, 2.213.749
Indonesia
-789.275, 113.921.327
India
20.593.684, 7.896.288
Laos
1.985.627, 102.495.496
USA
3.709.024, -95.712.891
Cuba
21.521.757, -77.781.167
España
40.463.667, -374.922
India
20.593.684, 7.896.288
Japan
36.204.824, 138.252.924
Island
64.963.051, -19.020.835
Cuba
21.521.757, -77.781.167
Israël
31.046.051, 34.851.612
Vietnam
14.058.324, 108.277.199
Australia
-25.274.398, 133.775.136
España
40.463.667, -374.922
India
20.593.684, 7.896.288
Italia
4.187.194, 1.256.738
Colombia
4.570.868, -74.297.333
Japan
36.204.824, 138.252.924
Brasil
-14.235.004, -5.192.528
Maldive
3.202.778, 7.322.068
India
20.593.684, 7.896.288
Maldives
3.202.778, 7.322.068
Brazil
-14.235.004, -5.192.528
Mexic
23.634.501, -102.552.784
Estados Unidos
3.709.024, -95.712.891
I